In [1]:
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import copy
import numpy as np
from torchvision import datasets, transforms
import torch

from utils.sampling import mnist_iid, mnist_noniid, cifar_iid, cifar_noniid, cifar_noniid_shared
from utils.options import args_parser
from models.Update import LocalUpdate
from models.Nets import MLP, CNNMnist, CNNCifar, LeNet, CNNMnist2
from models.Fed import FedAvg
from models.Fed import FedQAvg, Quantization, Quantization_Finite, my_score, my_score_Finite
from models.test import test_img


import math


# from sympy import * 
from utils.functions import *

%load_ext autoreload
%autoreload 2

In [2]:
class my_argument:    
    epochs = 200    #"rounds of training"
    num_users = 120  # "number of users: K"
    frac = 0.5 #"the fraction of clients: C"
    local_ep=1 #"the number of local epochs: E"
    local_bs=50 #"local batch size: B"
    bs=50 #"test batch size"
    lr=0.001 #"learning rate"
    momentum=0.9 # "SGD momentum (default: 0.5)"
    split='user' # "train-test split type, user or sample"
    weight_decay = 5e-4
    opt = 'SGD' #'ADAM'
    loss = 'Cross'

    # model arguments
    model = 'cnn'
    kernel_num=9 #, help='number of each kind of kernel')
    kernel_sizes='3,4,5' #  help='comma-separated kernel size to use for convolution')
    norm='batch_norm' #, help="batch_norm, layer_norm, or None")
    num_filters=32 #, help="number of filters for conv nets")
    max_pool='True' #help="Whether use max pooling rather than strided convolutions")

    # other arguments
    dataset='cifar' #, help="name of dataset")
    iid=1
    num_classes=10#, help="number of classes")
    num_channels=1#, help="number of channels of imges")
    gpu=1#, help="GPU ID, -1 for CPU")
    stopping_rounds=10#, help='rounds of early stopping')
    verbose='False'#, help='verbose print')
    seed=1#, help='random seed (default: 1)')
    
args = my_argument()

args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')

use_cuda = torch.cuda.is_available()
print(use_cuda)
args.device = torch.device("cuda" if use_cuda else "cpu")
print(args.device)

True
cuda


In [5]:
from torchvision import datasets, transforms


trans_cifar = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
dataset_train = datasets.CIFAR10('./data/cifar', train=True, download=True, transform=trans_cifar)
dataset_test = datasets.CIFAR10('./data/cifar', train=False, download=True, transform=trans_cifar)

dict_users = cifar_iid(dataset_train, args.num_users)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
N = args.num_users

p_array = np.array([0.1, 0.2, 0.3, 0.4, 0.5])
# p_array = np.array([0])

# print(p_matrix)
p_sel = np.random.randint(low=0, high=len(p_array), size=(N,))

p_per_user = np.ones((N,))

for i in range(N):
    p_per_user[i] = p_array[p_sel[i]]

print(p_per_user)

[0.1 0.3 0.1 0.5 0.1 0.1 0.3 0.1 0.5 0.5 0.1 0.4 0.5 0.3 0.4 0.4 0.5 0.1
 0.3 0.4 0.1 0.1 0.2 0.5 0.1 0.3 0.2 0.2 0.3 0.2 0.5 0.1 0.1 0.5 0.2 0.2
 0.2 0.5 0.3 0.2 0.2 0.5 0.2 0.2 0.3 0.4 0.1 0.5 0.4 0.1 0.1 0.1 0.2 0.5
 0.2 0.5 0.2 0.4 0.5 0.5 0.2 0.1 0.5 0.3 0.5 0.2 0.1 0.1 0.5 0.3 0.5 0.2
 0.5 0.2 0.2 0.3 0.5 0.5 0.5 0.4 0.1 0.2 0.1 0.1 0.5 0.5 0.2 0.2 0.4 0.2
 0.3 0.2 0.2 0.4 0.2 0.3 0.2 0.1 0.1 0.4 0.5 0.1 0.2 0.5 0.2 0.4 0.5 0.3
 0.2 0.1 0.1 0.1 0.1 0.3 0.4 0.5 0.5 0.5 0.3 0.1]


# 1. Random

In [18]:
from models.Nets import NIN,CNN_moderate, CNNCifar3
from models.vgg import *
from models.vggmodel import *

import math


lr_array = [0.001]


args.local_ep = 1
args.local_bs = 50
args.weight_decay = 3e-4

N = 40
K = 8

N_trials = 1
Max_iter = 200

args.opt = 'ADAM'

args.local_ep = 1
args.local_bs = 128


acc_test_arr_w_random_asymDrop  = np.zeros((len(lr_array), N_trials, Max_iter))
loss_test_arr_w_random_asymDrop = np.zeros((len(lr_array), N_trials, Max_iter))


for trial_idx in range(N_trials):
    
    for lr_idx in range(len(lr_array)):
        
        args.lr = lr_array[lr_idx]
        
        P_w_random_asymDrop = []
        
        print()
        print('Learning Rate =',args.lr)
        print()
#         net_glob = vgg11()
        net_glob = VGG_origin('VGG11')
#         net_glob = VGG('VGG11')

        net_glob = net_glob.cuda()
        print(net_glob)

        net_glob.train()

        # copy weights
        w_glob = net_glob.state_dict()
        for iter in range(Max_iter): #args.epochs
            w_locals, loss_locals = [], []
            
            if iter == 400 or iter == 800:
                args.lr = args.lr * 0.4
                
                
            ###############################
            # 0. Dropout Realization
            ###############################    
            
            u = np.ones((N,))
            for u_idx in range(N):
                p_sel = p_per_user[u_idx]
                u[u_idx] = np.random.binomial(1, 1-p_sel, size=1)[0]
            
            result = np.where(u == 1)
            drop_result = np.where(u == 0)

            ###############################
            # 1. Weighted Random Selection
            ###############################

            if iter == 0:
                idxs_users = np.random.choice(result[0], K, replace=False)
#                 print('select=',select)
            else:
                P = np.array(P_w_random_asymDrop)
                P_sum = np.sum(P, axis=0).astype(int)
        
                for i in drop_result[0]:
                    P_sum[i] = Max_iter + 1   

                P_sum_sort = P_sum.argsort()

                idxs_users = P_sum_sort[:K]
            

            p_tmp = np.zeros(N)
            p_tmp[idxs_users] = 1

            P_w_random_asymDrop.append(p_tmp)


        #     idxs_users = np.random.choice(range(N), K, replace=False)
            for idx in idxs_users:
        #         print(idx)
                local = LocalUpdate(args=args, dataset=dataset_train, idxs=dict_users[idx])
                w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))
                w_locals.append(copy.deepcopy(w))
                loss_locals.append(copy.deepcopy(loss))
            # update global weights
            w_glob = FedAvg(w_locals)

            # copy weight to net_glob
            net_glob.load_state_dict(w_glob)

            # print loss
            loss_avg = sum(loss_locals) / len(loss_locals)

    #         loss_train.append(loss_avg)

            acc_test, loss_test = test_img(net_glob, dataset_test, args)
            acc_test_arr_w_random_asymDrop[lr_idx][trial_idx][iter]  = acc_test
            loss_test_arr_w_random_asymDrop[lr_idx][trial_idx][iter] = loss_test
            if iter % 1 ==0:
                print('Round {:3d}, Train average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_avg,acc_test))
            #print(loss_train)


Learning Rate = 0.001

VGG_origin(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inpla


Test set: Average loss: 1.1382 
Accuracy: 5901/10000 (59.01%)

Round  50, Train average loss 1.173 Test accuracy 59.010

Test set: Average loss: 0.9933 
Accuracy: 6430/10000 (64.30%)

Round  51, Train average loss 1.161 Test accuracy 64.300

Test set: Average loss: 1.0098 
Accuracy: 6399/10000 (63.99%)

Round  52, Train average loss 1.123 Test accuracy 63.990

Test set: Average loss: 1.0756 
Accuracy: 6133/10000 (61.33%)

Round  53, Train average loss 1.186 Test accuracy 61.330

Test set: Average loss: 1.0359 
Accuracy: 6292/10000 (62.92%)

Round  54, Train average loss 1.125 Test accuracy 62.920

Test set: Average loss: 1.0521 
Accuracy: 6313/10000 (63.13%)

Round  55, Train average loss 1.133 Test accuracy 63.130

Test set: Average loss: 1.0811 
Accuracy: 6088/10000 (60.88%)

Round  56, Train average loss 1.093 Test accuracy 60.880

Test set: Average loss: 1.0955 
Accuracy: 6143/10000 (61.43%)

Round  57, Train average loss 1.099 Test accuracy 61.430

Test set: Average loss: 1.1485 


Test set: Average loss: 1.1079 
Accuracy: 6933/10000 (69.33%)

Round 118, Train average loss 0.707 Test accuracy 69.330

Test set: Average loss: 1.1082 
Accuracy: 7000/10000 (70.00%)

Round 119, Train average loss 0.671 Test accuracy 70.000

Test set: Average loss: 1.1881 
Accuracy: 6765/10000 (67.65%)

Round 120, Train average loss 0.669 Test accuracy 67.650

Test set: Average loss: 1.0386 
Accuracy: 7054/10000 (70.54%)

Round 121, Train average loss 0.610 Test accuracy 70.540

Test set: Average loss: 1.1572 
Accuracy: 6883/10000 (68.83%)

Round 122, Train average loss 0.656 Test accuracy 68.830

Test set: Average loss: 0.9779 
Accuracy: 7198/10000 (71.98%)

Round 123, Train average loss 0.623 Test accuracy 71.980

Test set: Average loss: 0.9962 
Accuracy: 7200/10000 (72.00%)

Round 124, Train average loss 0.591 Test accuracy 72.000

Test set: Average loss: 1.0163 
Accuracy: 7077/10000 (70.77%)

Round 125, Train average loss 0.665 Test accuracy 70.770

Test set: Average loss: 1.0206 


Test set: Average loss: 1.4190 
Accuracy: 6852/10000 (68.52%)

Round 186, Train average loss 0.451 Test accuracy 68.520

Test set: Average loss: 1.2811 
Accuracy: 7048/10000 (70.48%)

Round 187, Train average loss 0.371 Test accuracy 70.480

Test set: Average loss: 1.2034 
Accuracy: 7076/10000 (70.76%)

Round 188, Train average loss 0.517 Test accuracy 70.760

Test set: Average loss: 1.3410 
Accuracy: 6981/10000 (69.81%)

Round 189, Train average loss 0.406 Test accuracy 69.810

Test set: Average loss: 1.3051 
Accuracy: 6950/10000 (69.50%)

Round 190, Train average loss 0.398 Test accuracy 69.500

Test set: Average loss: 1.1477 
Accuracy: 7337/10000 (73.37%)

Round 191, Train average loss 0.397 Test accuracy 73.370

Test set: Average loss: 1.2938 
Accuracy: 7042/10000 (70.42%)

Round 192, Train average loss 0.443 Test accuracy 70.420

Test set: Average loss: 1.1590 
Accuracy: 7246/10000 (72.46%)

Round 193, Train average loss 0.394 Test accuracy 72.460

Test set: Average loss: 1.1847 